# Homework 1 - Time Series

This notebook contains all code required for Homework 1.
Use the markdown sections to add your written analysis and conclusions.


In [ ]:
%pip install -q pandas matplotlib seaborn xlrd openpyxl


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 4)


## Part 1: Impact of September 11 on U.S. Travel


In [ ]:
travel_path = "../resources/Sept11Travel.xls"

try:
    travel = pd.read_excel(travel_path)
except Exception:
    # Fallback in case file extension/engine differs on another machine
    travel = pd.read_excel(travel_path, engine="xlrd")

travel.columns = [str(col).strip() for col in travel.columns]
travel.head()


In [ ]:
# Build a monthly date index if one is not already present
if "Month" in travel.columns:
    travel["Month"] = pd.to_datetime(travel["Month"])
    travel = travel.set_index("Month")
elif "Date" in travel.columns:
    travel["Date"] = pd.to_datetime(travel["Date"])
    travel = travel.set_index("Date")
else:
    travel.index = pd.date_range(start="1990-01-01", periods=len(travel), freq="MS")

# Normalize expected column names
rename_map = {}
for col in travel.columns:
    lower = col.lower()
    if "air" in lower:
        rename_map[col] = "Air"
    elif "rail" in lower:
        rename_map[col] = "Rail"
    elif "car" in lower or "vehicle" in lower:
        rename_map[col] = "Car"

travel = travel.rename(columns=rename_map)
series_cols = [c for c in ["Air", "Rail", "Car"] if c in travel.columns]
travel = travel[series_cols].dropna()
travel.head()


In [ ]:
# Plot each of the three original series
fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(12, 10))
for ax, col in zip(axes, series_cols):
    ax.plot(travel.index, travel[col], color="#1f77b4", linewidth=2)
    ax.set_title(f"{col} - Original Scale")
    ax.set_ylabel("Passenger / Miles")

axes[-1].set_xlabel("Date")
plt.tight_layout()
plt.show()


### Analysis space (original-scale trends)

Write your interpretation of the trend behavior here.


In [ ]:
# Rescale series to compare shapes directly (z-score)
scaled = (travel - travel.mean()) / travel.std()

fig, ax = plt.subplots(figsize=(12, 5))
for col in scaled.columns:
    ax.plot(scaled.index, scaled[col], linewidth=2, label=col)

# Add linear trendlines
x = np.arange(len(scaled))
for col in scaled.columns:
    y = scaled[col].values
    coeffs = np.polyfit(x, y, deg=1)
    trend = np.poly1d(coeffs)(x)
    ax.plot(scaled.index, trend, linestyle="--", linewidth=1.8, label=f"{col} trend")

ax.set_title("Scaled Travel Series with Linear Trendlines")
ax.set_ylabel("Standardized value (z-score)")
ax.set_xlabel("Date")
ax.legend(ncol=2, bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.show()


### Analysis space (scaled series + trendlines)

Write any new observations you notice after scaling and adding trendlines here.


## Part 2: Forecasting Department Store Sales


In [ ]:
sales_path = "../resources/DepartmentStoreSales.csv"
sales = pd.read_csv(sales_path)
sales.columns = [str(c).strip() for c in sales.columns]
sales.head()


In [ ]:
# Build a quarterly datetime index
if "Quarter" in sales.columns:
    sales["Quarter"] = sales["Quarter"].astype(str).str.replace(" ", "", regex=False)
    # Accept formats like Q1-2018 or 2018 Q1
    q = sales["Quarter"].str.extract(r"(?:(Q[1-4]).*?(\d{4})|(\d{4}).*?(Q[1-4]))")
    q_label = q[0].fillna(q[3])
    year = q[1].fillna(q[2])
    sales_index = pd.PeriodIndex(year + q_label, freq="Q").to_timestamp()
    sales = sales.set_index(sales_index)
elif "Date" in sales.columns:
    sales["Date"] = pd.to_datetime(sales["Date"])
    sales = sales.set_index("Date")
else:
    sales.index = pd.period_range(start="2000Q1", periods=len(sales), freq="Q").to_timestamp()

# Identify sales column
sales_col = None
for c in sales.columns:
    if "sale" in c.lower():
        sales_col = c
        break
if sales_col is None:
    sales_col = sales.columns[-1]

sales_series = sales[sales_col].astype(float)
sales_series.head()


In [ ]:
# Well-formatted quarterly sales plot
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(sales_series.index, sales_series.values, marker="o", linewidth=2, color="#2ca02c")
ax.set_title("Department Store Quarterly Sales")
ax.set_xlabel("Quarter")
ax.set_ylabel("Sales")
ax.grid(alpha=0.25)
plt.tight_layout()
plt.show()


In [ ]:
# Quick decomposition-style helpers for trend/seasonality discussion
sales_df = sales_series.to_frame(name="sales")
sales_df["rolling_mean_4"] = sales_df["sales"].rolling(window=4).mean()
sales_df["detrended"] = sales_df["sales"] - sales_df["rolling_mean_4"]

fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
axes[0].plot(sales_df.index, sales_df["sales"], marker="o", label="Sales")
axes[0].plot(sales_df.index, sales_df["rolling_mean_4"], linestyle="--", label="4-quarter rolling mean")
axes[0].set_title("Sales and Smoothed Trend")
axes[0].legend()

axes[1].plot(sales_df.index, sales_df["detrended"], marker="o", color="#d62728")
axes[1].axhline(0, color="black", linewidth=1)
axes[1].set_title("Detrended Series (for seasonality/noise inspection)")
axes[1].set_xlabel("Quarter")

plt.tight_layout()
plt.show()


### Analysis space (trend, seasonality, noise)

Discuss which components seem present in the quarterly sales series and why.
